# Quickstart Neo4j

Let's download and visualize the graph data about communities which is hosted on [Neo4j](https://neo4j.com/docs/). Neo4j can be querried with the [Cypher querrying language](https://neo4j.com/docs/cypher/).

## Downloading data from Neo4j 

Here is an example of using the Neo4JDownloader to retrieve all nodes from the graph. 
Adapt the Neo4JDownloader class as you see fit for retrieving information. 

In [ ]:
# Define your NEO4J database

neo4j_database = "neo4j"  

# Define your nodes

nodes = ["user", "repo", "org"]

# Define your relationships (edges)

relationships = {
    "member_of": {"type1": {"source": "user", "target": "org"}},
    "owner_of": {
        "type1": {"source": "user", "target": "repo"},
        "type2": {"source": "org", "target": "repo"},
    },
    "contributor_of": {
        "type1": {"source": "user", "target": "repo"},
        "type2": {"source": "org", "target": "repo"},
    },
    "parent_of": {
        "type1": {"source": "repo", "target": "repo"},
    },
}

In [ ]:
import neo4j
from utils.neo4jdownloader import Neo4JDownloader
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

def get_downloader(neo4j_database):
    NEO4J_URI = os.environ.get("NEO4J_URI")
    NEO4J_USERNAME = os.environ.get("NEO4J_USER")
    NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")
    NEO4J_DATABASE = neo4j_database

    return Neo4JDownloader(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_DATABASE)

def extract_data(neo4j_database, nodes, relationships):
    downloader = get_downloader(neo4j_database)

    try:
        nodes_ids, nodes_features = downloader.retrieve_nodes(nodes)
        edges_indices, edges_attributes = downloader.retrieve_edges(relationships)

        return nodes_ids, nodes_features, edges_indices, edges_attributes
    finally:
        downloader.close()

In [ ]:
nodes_ids, nodes_features, edges_indices, edges_attributes = extract_data(neo4j_database, nodes, relationships)
# example of looking at the output
# print(nodes_ids["org"])
# print(nodes_features["org"])
# print(edges_indices)

In [ ]:
from utils.builder_dataframe import neo4j_to_dataframe
df = neo4j_to_dataframe(nodes_ids, nodes_features, edges_indices, relationships)
df.head()

## Visualizing the Graph

Here is some code snippet for visualizing the graph you have downloaded from Neo4J.